In [1]:
!pip install wget

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from subprocess import call
from scipy.stats.distributions import gamma,lognorm
import json 
import wget
import os
import os.path
import OrderedDict
# from google.colab import drive
# os.chdir('/content/gdrive/My Drive')
# drive.mount('/content/gdrive')

In [3]:
def n2z(x):
    x[np.logical_or(np.isnan(x),np.isinf(x))] = 0
    return x

if os.path.exists(os.getcwd()+"\\national.json"):
    os.remove(os.getcwd()+"\\national.json")
wget.download('https://api.covid19india.org/data.json', os.getcwd()+"//national.json")

if os.path.exists(os.getcwd()+"\\states.json"):
    os.remove(os.getcwd()+"\\states.json")
wget.download('https://api.covid19india.org/states_daily.json', os.getcwd()+"//states.json")

if os.path.exists(os.getcwd()+"\\test.json"):
    os.remove(os.getcwd()+"\\test.json")
wget.download('https://api.covid19india.org/state_test_data.json', os.getcwd()+"//test.json")

'/home/nbuser/library//test (1).json'

In [4]:
dataset=pd.read_csv('./population.csv')

In [5]:
population=pd.DataFrame()
population["State"]=dataset['State'][:37]
population["Population"]=dataset['Population'][:37]
population

,State,Population
0,Delhi,18710922
1,Haryana,28204692
2,Kerala,35699443
3,Himachal Pradesh,7451955
4,Punjab,30141373
5,Telangana,39362732
6,Goa,1586250
7,Uttarakhand,11250858
8,Chandigarh,1158473
9,Gujarat,63872399


In [6]:
def fn(mon):
  if(mon == "01"):
    return " January"
  if(mon == "02"):
    return " February"
  if(mon == "03"):
    return " March"
  if(mon == "04"):
    return " April"
  if(mon == "05"):
    return " May"
  if(mon == "06"):
    return " June"
  if(mon == "07"):
    return " July"
  if(mon == "08"):
    return " August"
  if(mon == "09"):
    return " September"
  if(mon == "10"):
    return " October"
  if(mon == "11"):
    return " November"
  if(mon == "12"):
    return " December"
def convert(dat): 
    return  str(dat[:2]) + fn(str(dat[3:5]))

In [7]:
dates = np.array([pd.to_datetime(i['date']+"2020") for i in json.load(open('national.json',))['cases_time_series']])
confirmed = np.array([int(i['dailyconfirmed'])for i in json.load(open('national.json',))['cases_time_series']])
deceased = np.array([int(i['dailydeceased'])for i in json.load(open('national.json',))['cases_time_series']])
recovered = np.array([int(i['dailyrecovered'])for i in json.load(open('national.json',))['cases_time_series']])

confirmed = confirmed[dates>=pd.to_datetime("03/04/20")]
deceased = deceased[dates>=pd.to_datetime("03/04/20")]
recovered = recovered[dates>=pd.to_datetime("03/04/20")]
dates = dates[dates>=pd.to_datetime("03/04/20")]

In [8]:
conf = []

for n in range(100):
    print("Iteration: ",n+1,end='\r')
    dataset = np.copy(confirmed)
    mean = 13.0+(20.9-8.7)/4*np.random.normal()
    sd = 12.7+(26.0-6.4)/4*np.random.normal()
    phi = np.sqrt(sd**2 + mean**2)
    mu = np.log(mean**2/phi)
    sigma = np.sqrt(np.log(phi**2/mean**2))
    L = lognorm(s=sigma,scale=np.exp(mu))
    for i in range(len(dataset)-1,-1,-1):
        send_forward = np.round(L.rvs(dataset[i]))
        send_forward = send_forward[i+send_forward<len(dataset)]
        dataset[i] = 0
        for j in np.unique(np.int32(send_forward)):
            dataset[i+j] += np.sum(send_forward==j)
    conf.append(dataset)

In [9]:
CFR = np.cumsum(deceased)/np.cumsum(conf,axis=1)
col_mean = np.nanmean(CFR, axis=0)
inds = np.where(np.isnan(CFR))
CFR[inds] = np.take(col_mean, inds[1])

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [10]:
temp=dates
dates=[]
for i in range(len(temp)):
  date=[]
  t=(str(temp[i])).split()[0]
  m=t[5:7]
  d=t[-2:]
  #print(m,t)
  date.append(d)
  date.append('/')
  date.append(m)
  date=''.join([str(elem) for elem in date])
  #print(date)
  dates.append(convert(date))

In [11]:
list(100*n2z(np.quantile(CFR,0.025,axis=0)))

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.3518826135105204,
 2.0820668693009114,
 3.6013645224171538,
 3.2239583333333335,
 2.917910447761194,
 3.74765478424015,
 3.1901528013582343,
 3.702306439166595,
 3.149606299212598,
 2.475258918296893,
 3.644982774055286,
 3.7713675213675217,
 3.376525435348965,
 3.2352941176470584,
 3.9925145031501463,
 4.1472908869872,
 4.450549450549451,
 4.223080145313445,
 5.617406871405481,
 5.594032744506678,
 5.183450395713937,
 5.527762885882225,
 5.418676962110225,
 5.183963905476064,
 5.273484193767569,
 5.093085025580098,
 5.273481379559245,
 5.157005726145693,
 5.710139652779221,
 5.510229591566549,
 5.6614760576316465,
 5.789798914850451,
 5.599467847275276,
 5.505194598186949,
 5.298498476669907,
 5.103116932930493,
 5.059911413778928,
 4.9798601536546725,
 4.880889022646259,
 4.736264538079082,
 4.73948900129285,
 4.62637093503397,
 4.539438619333214,
 4.5554095919346675,
 4.4741676442706435,
 4.448758157488114,
 4.4339101041102165,
 4.4

In [12]:
json_data = {}
cfr1_point=list(100*n2z(np.cumsum(deceased)/np.cumsum(confirmed))),
cfr2_point=list(100*n2z(np.cumsum(deceased)/(np.cumsum(deceased)+np.cumsum(recovered)))),
cfr3_point=list(100*n2z(np.mean(CFR,axis=0))),
cfr3_l95=list(100*n2z(np.quantile(CFR,0.025,axis=0))),
cfr3_u95=list(100*n2z(np.quantile(CFR,0.975,axis=0))),
cfr3_l50=list(100*n2z(np.quantile(CFR,0.25,axis=0))),
cfr3_u50=list(100*n2z(np.quantile(CFR,0.75,axis=0))),
# india = {
#         'dates':dates,
#         'cfr1_point':list(100*n2z(np.cumsum(deceased)/np.cumsum(confirmed))),
#         'cfr2_point':list(100*n2z(np.cumsum(deceased)/(np.cumsum(deceased)+np.cumsum(recovered)))),
#         'cfr3_point':list(100*n2z(np.mean(CFR,axis=0))),
#         'cfr3_l95':list(100*n2z(np.quantile(CFR,0.025,axis=0))),
#         'cfr3_u95':list(100*n2z(np.quantile(CFR,0.975,axis=0))),
#         'cfr3_l50':list(100*n2z(np.quantile(CFR,0.25,axis=0))),
#         'cfr3_u50':list(100*n2z(np.quantile(CFR,0.75,axis=0))),
#         }
india = OrderedDict.OrderedDict([('dates',dates),('cfr1_point',cfr1_point),('cfr2_point',cfr2_point),('cfr3_point',cfr3_point),('cfr3_l95',cfr3_l95),('cfr3_u95',cfr3_u95),('cfr3_l50',cfr3_l50),('cfr3_u50',cfr3_u50)])
json_data['India'] = india
cfr = pd.DataFrame()
cfr['state']=['India']*len(dates)
cfr['dates']=dates
cfr['cfr1_point']=list(100*n2z(np.cumsum(deceased)/np.cumsum(confirmed)))
cfr['cfr2_point']=list(100*n2z(np.cumsum(deceased)/(np.cumsum(deceased)+np.cumsum(recovered))))
cfr['cfr3_point']=list(100*n2z(np.mean(CFR,axis=0)))
cfr['cfr3_l95']=list(100*n2z(np.quantile(CFR,0.025,axis=0)))
cfr['cfr3_u95']=list(100*n2z(np.quantile(CFR,0.975,axis=0)))
cfr['cfr3_l50']=list(100*n2z(np.quantile(CFR,0.25,axis=0)))
cfr['cfr3_u50']=list(100*n2z(np.quantile(CFR,0.75,axis=0)))
cfr

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in true_divide
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in true_divide


,state,dates,cfr1_point,cfr2_point,cfr3_point,cfr3_l95,cfr3_u95,cfr3_l50,cfr3_u50
0,India,04 March,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,India,05 March,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,India,06 March,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,India,07 March,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,India,08 March,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,India,09 March,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,India,10 March,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,India,11 March,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,India,12 March,1.333333,50.000000,0.000000,2.351883,0.000000,3.961538,16.666667
9,India,13 March,1.176471,12.500000,0.000000,2.082067,100.000000,3.333333,11.111111


In [13]:
states = list(filter(lambda v:len(v)<3,list(json.load(open('states.json',))['states_daily'][0].keys())))

In [14]:
dates = np.array([pd.to_datetime(i['date']) for i in filter(lambda v: v['status'] == 'Confirmed',json.load(open('states.json',))['states_daily'])])
#print(dates)
data_recovered = pd.DataFrame()
data_deceased = pd.DataFrame()
data_confirmed = pd.DataFrame()
for st in states:
    data_confirmed[st] = np.array([i[st] for i in filter(lambda v: v['status'] == 'Confirmed',json.load(open('states.json',))['states_daily'])])
    data_deceased[st] = np.array([i[st] for i in filter(lambda v: v['status'] == 'Deceased',json.load(open('states.json',))['states_daily'])])
    data_recovered[st] = np.array([i[st] for i in filter(lambda v: v['status'] == 'Recovered',json.load(open('states.json',))['states_daily'])])

In [15]:
data_recovered = data_recovered.replace(r'^\s*$', np.NaN, regex=True).fillna(0)
data_recovered = data_recovered.astype(np.int32)
data_confirmed = data_confirmed.replace(r'^\s*$', np.NaN, regex=True).fillna(0)
data_confirmed = data_confirmed.astype(np.int32)
data_deceased = data_deceased.replace(r'^\s*$', np.NaN, regex=True).fillna(0)
data_deceased = data_deceased.astype(np.int32)
data_deceased['date'] = dates
data_recovered['date'] = dates
data_confirmed['date'] = dates

In [16]:
final=pd.DataFrame
plt.figure(1, figsize=(15, 7))
state_id = {
  "mh":"Maharashtra",
  "tn":"Tamil Nadu",
  "dl":"Delhi",
  "gj":"Gujarat",
  "rj":"Rajasthan",
  "up":"Uttar Pradesh",
  "mp":"Madhya Pradesh",
  "wb":"West Bengal",
  "ka":"Karnataka",
  "br":"Bihar",
  "ap":"Andhra Pradesh",
  "hr":"Haryana",
  "tg":"Telangana",
  "jk":"Jammu and Kashmir",
  "or":"Odisha",
  "pb":"Punjab",
  "as":"Assam",
  "kl":"Kerala",
  "ut":"Uttarakhand",
  "jh":"Jharkand",
  "ct":"Chhattisgarh",
  "tr":"Tripura",
  "hp":"Himachal Pradesh",
  "ch":"Chandigarh",
  "ga":"Goa",
  "mn":"Manipur",
  "nl":"Nagaland",
  "py":"Puducherry",
  "la":"Ladakh",
  "ar":"Arunachal Pradesh",
  "an":"Andaman and Nicobar Islands",
  "ml":"Meghalaya",
  "mz":"Mizoram",
  "dn":"Dadra and Nagar Haveli and Daman and Diu",
  "sk":"Sikkim",
}

for state in state_id.keys():
    boots = 100
    conf = []
    for n in range(boots):
        #print("Iteration: ",n+1,end='\r')
        dataset = np.copy(data_confirmed[state].values)
        mean = 13.0+(20.9-8.7)/4*np.random.normal()
        sd = 12.7+(26.0-6.4)/4*np.random.normal()
        phi = np.sqrt(sd**2 + mean**2)
        mu = np.log(mean**2/phi)
        sigma = np.sqrt(np.log(phi**2/mean**2))
        L = lognorm(s=sigma,scale=np.exp(mu))
        for i in range(len(dataset)-1,-1,-1):
            send_forward = np.round(L.rvs(np.max([dataset[i],0])))
            send_forward = send_forward[i+send_forward<len(dataset)]
            dataset[i] = 0
            for j in np.unique(np.int32(send_forward)):
                dataset[i+j] += np.sum(send_forward==j)
        conf.append(dataset)
    CFR = np.cumsum(data_deceased[state].values)/np.cumsum(conf,axis=1)
    col_mean = np.nanmean(CFR, axis=0)
    inds = np.where(np.isnan(CFR))
    CFR[inds] = np.take(col_mean, inds[1])
    temp1=list(pd.Series(dates).dt.strftime('%m-%d-%Y'))
    #print(temp1[0])
    dates1=[]
    for i in range(len(temp1)):
      date=temp1[i][3:5]+'/'+temp1[i][0:2]
      dates1.append(convert(date))
    temp = {
        'dates':dates1,
        'cfr1_point':list(n2z(100*np.cumsum(data_deceased[state].values)/np.cumsum(data_confirmed[state].values))),
        'cfr2_point':list(n2z(100*np.cumsum(data_deceased[state].values)/(np.cumsum(data_deceased[state].values)+np.cumsum(data_recovered[state].values)))),
        'cfr3_point':list(n2z(100*np.mean(CFR,axis=0))),
        'cfr3_l95':list(n2z(100*np.quantile(CFR,0.025,axis=0))),
        'cfr3_u95':list(n2z(100*np.quantile(CFR,0.975,axis=0))),
        'cfr3_l50':list(n2z(100*np.quantile(CFR,0.25,axis=0))),
        'cfr3_u50':list(n2z(100*np.quantile(CFR,0.75,axis=0))),
        }
    a=state_id[state]
    #print(a)
    json_data[str(a)] = temp
    cfr_state=pd.DataFrame()
    cfr_state['state']=[str(a)]*len(dates)
    cfr_state['dates']=dates1
    cfr_state['cfr1_point']=(list(100*n2z(np.cumsum(data_deceased[state].values)/np.cumsum(data_confirmed[state].values))))
    cfr_state['cfr2_point']=(list(100*n2z(np.cumsum(data_deceased[state].values)/(np.cumsum(data_deceased[state].values)+np.cumsum(data_recovered[state].values)))))
    cfr_state['cfr3_point']=(list(100*n2z(np.mean(CFR,axis=0))))
    cfr_state['cfr3_l95']=(list(100*n2z(np.quantile(CFR,0.025,axis=0))))
    cfr_state['cfr3_u95']=(list(100*n2z(np.quantile(CFR,0.975,axis=0))))
    cfr_state['cfr3_l50']=(list(100*n2z(np.quantile(CFR,0.25,axis=0))))
    cfr_state['cfr3_u50']=(list(100*n2z(np.quantile(CFR,0.75,axis=0))))
    cfr=pd.concat([cfr, cfr_state])
      
    #plt.plot(temp['cfr3_point'],label=state)
#plt.legend()

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:61: RuntimeWarning: Mean of empty slice
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:72: RuntimeWarning: invalid value encountered in true_divide
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in true_divide
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:86: RuntimeWarning: invalid value encountered in true_divide
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:87: RuntimeWarning: invalid value encountered in true_divide
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:60: RuntimeWarning: divide by zero encountered in true_divide


In [17]:
print(list(100*n2z(np.mean(CFR,axis=0))))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.677239419544568, 6.8593764796621794, 5.866337311900644, 5.383932290072172, 4.8942983901979975, 4.599077418008017, 4.301909584671058, 4.112912825423691, 3.955045568421911, 3.81997560702775, 3.681509950001025, 3.5895343968929154, 3.4940826365716853, 3.406838642864877, 3.3478011907817096, 3.284280153019517, 3.2194551513467924, 3.163223694953612, 3.0915615883458134, 3.02441380849773, 2.963409681695714, 2.9024122010292968, 2.8518729007155224, 2.792279551168067, 2.7334044418817633, 5.302764737059733, 2.568610864500229, 2.4856409228341176, 2.3943396275177173, 2.3115871963944357, 2.2263975007522747, 2.139260981944212, 2.0292036258010784, 1.9310083841180274, 1.8378971979319374, 5.241906135500221, 4.941775029695655, 4.586173420091069, 4.187000092323848, 3.7941632118386166, 3.4219447763919475, 3.047624269680323, 2.6640630119771442, 2.304849119714

In [18]:
cfr.to_csv('cfr.csv',index=False)
from datetime import datetime
json_data['datetime']=str(datetime.now())

In [19]:
json_data_indented = json.dumps(json_data, indent = 4)
with open("cfr.json", "w") as outfile: 
    outfile.write(json_data_indented)

In [20]:
total_confirmed = [x['totalconfirmed'].split(' ')[0] for x in json.load(open('national.json',))['cases_time_series']] ## x_cumulative= 'totalconfirmed'
daily_confirmed = [x['dailyconfirmed'].split(' ')[0] for x in json.load(open('national.json',))['cases_time_series']] ## x1
daily_confirmed_ma=['']*7
dates=[]
daily_confirmed=list(map(int,daily_confirmed))
for i in range(7,len(daily_confirmed)):
    daily_confirmed_ma.append(sum(daily_confirmed[i-7:i])/7)
datesspace = [x['date'] for x in json.load(open('national.json',))['cases_time_series']]
for i in range(len(datesspace)):
    dates.append(datesspace[i][0:-1])
#print(total_confirmed)

In [21]:
datesspace

['30 January ',
 '31 January ',
 '01 February ',
 '02 February ',
 '03 February ',
 '04 February ',
 '05 February ',
 '06 February ',
 '07 February ',
 '08 February ',
 '09 February ',
 '10 February ',
 '11 February ',
 '12 February ',
 '13 February ',
 '14 February ',
 '15 February ',
 '16 February ',
 '17 February ',
 '18 February ',
 '19 February ',
 '20 February ',
 '21 February ',
 '22 February ',
 '23 February ',
 '24 February ',
 '25 February ',
 '26 February ',
 '27 February ',
 '28 February ',
 '29 February ',
 '01 March ',
 '02 March ',
 '03 March ',
 '04 March ',
 '05 March ',
 '06 March ',
 '07 March ',
 '08 March ',
 '09 March ',
 '10 March ',
 '11 March ',
 '12 March ',
 '13 March ',
 '14 March ',
 '15 March ',
 '16 March ',
 '17 March ',
 '18 March ',
 '19 March ',
 '20 March ',
 '21 March ',
 '22 March ',
 '23 March ',
 '24 March ',
 '25 March ',
 '26 March ',
 '27 March ',
 '28 March ',
 '29 March ',
 '30 March ',
 '31 March ',
 '01 April ',
 '02 April ',
 '03 April ',

In [22]:
dates_dict = {}
count=1
for i in dates:
  if i not in dates_dict.keys():
    dates_dict[i]=count
    count+=1
print(dates_dict)

{'16 May': 108, '05 February': 7, '11 April': 73, '09 February': 11, '25 March': 56, '17 February': 19, '11 February': 13, '13 May': 105, '10 April': 72, '18 May': 110, '09 March': 40, '30 April': 92, '05 April': 67, '24 February': 26, '20 February': 22, '20 April': 82, '01 June': 124, '24 March': 55, '02 February': 4, '08 February': 10, '31 March': 62, '11 March': 42, '08 May': 100, '07 April': 69, '18 February': 20, '07 March': 38, '27 April': 89, '31 May': 123, '13 March': 44, '03 June': 126, '13 February': 15, '03 March': 34, '09 May': 101, '06 June': 129, '02 April': 64, '12 April': 74, '02 March': 33, '23 April': 85, '21 March': 52, '10 May': 102, '21 May': 113, '06 March': 37, '22 March': 53, '10 June': 133, '29 March': 60, '02 June': 125, '22 April': 84, '03 May': 95, '12 March': 43, '06 May': 98, '29 April': 91, '09 April': 71, '05 March': 36, '07 May': 99, '23 May': 115, '28 April': 90, '10 March': 41, '27 May': 119, '01 May': 93, '31 January': 2, '15 April': 77, '13 April': 

In [23]:
def convert(dat): 
    return  str(dat[:2]) + fn(str(dat[3:5]))

In [24]:
population['Population'][36]

1371360350

In [25]:
datest=[x['updatetimestamp'].split(' ')[0] for x in json.load(open('national.json',))['tested']]
#totalcumulative_tested = [] #y_cumulative
y_cumulative1=[x['totalsamplestested'] for x in json.load(open('national.json',))['tested']]
test_per_million=['']*len(dates)
dates1=[]
for i in (datest):
  dates1.append(convert(i))
datestest = []
datestest.append(dates1[0])
y_cumulative = []
tested_cum=['']*len(dates)
y_cumulative.append(y_cumulative1[0])
### y = 'totalsamplestested'
#daily_tested = [] # y1 (y_cumulative2-ycumulative1)
for i in range(1,len(dates1)):
  if (dates1[i]==dates1[i-1]):
    y_cumulative[-1]=y_cumulative1[i]
  else:
    datestest.append(dates1[i])
    y_cumulative.append(y_cumulative1[i])
daily_tested=[]
daily_tested_final=['']*len(dates)
daily_confirmed_final=['']*len(dates)
daily_confirmed_ma_final=['']*len(dates)
daily_tested.append(y_cumulative[0])
for i in range(1,len(y_cumulative)):
  if (len(y_cumulative[i])!=0):
    a=int(y_cumulative[i])-int(y_cumulative[i-1])
    daily_tested.append(a)
    #test_per_million.append(int(y_cumulative[i])*1000000/int(population['Population'][36]))
  else:
    y_cumulative[i]=y_cumulative[i-1]
    daily_tested.append(0)
    #test_per_million.append('')
var=0
pos_rate_cum=['']*len(dates)
daily_pos_rate=['']*len(dates)
for i in range(len(datestest)):
  for j in range(var,len(dates)):
    if (datestest[i]==dates[j]):
      if (int(daily_tested[i])!=0):
        daily_pos_rate[j]=(int(daily_confirmed[j])*100/int(daily_tested[i]))
        pos_rate_cum[j]=(int(total_confirmed[j])*100/int(y_cumulative[i]))
        test_per_million[j]=(int(y_cumulative[i])*1000000/int(population['Population'][36]))
        daily_tested_final[j]=daily_tested[i]
        tested_cum[j]=y_cumulative[i]
      else:
        daily_pos_rate[j]=('')
        #test_per_million.append('')
      var=j
      break
#print(len(y_cumulative))
#daily_confirmed_7day_moving = []
## India : date : []
##        cumulative pos rate : [] x_cum/y_cum
##        daily_pos_rate : []  (after taking moving avg) -> 7 observation moving avg
## daily confirmed : [] (7 day moving average)

In [26]:
print(len(daily_confirmed_ma))

134


In [27]:
len(daily_pos_rate)

134

In [28]:
daily_pos_rate_ma=['']*len(dates)
from statistics import mean
count=0
temp=['']*len(dates)

i=0
l=len(dates)
#print(datestest)
# for i in range(len(dates)):
#print(daily_pos_rate)

while i<l:
  # print(i)
  count+=1
  for j in range(len(datestest)):
    if (dates[i]==datestest[j]):
      a=daily_pos_rate[i]
      temp[i]=str(a)
  
  for k in range(7,len(temp)):
    sum=0
    counter=0
    for m in range(0,7):
      if (len(str(temp[k-m]))!=0):
        sum+=float(temp[k-m])
      else:
        counter+=1
    if ((7-counter)!=0):
      daily_pos_rate_ma[k]=(sum/(7-counter))
     
  i=i+1

In [29]:
india_dates = dates
india_total_cases=total_confirmed
india_daily_confirmed = daily_confirmed
india_daily_confirmed_ma = daily_confirmed_ma
india_total_confirmed = total_confirmed
india_daily_pos_rate_ma = daily_pos_rate_ma
india_pos_rate_cum = pos_rate_cum
india_daily_pos_rate = daily_pos_rate
india_test_per_million=test_per_million
india_daily_tested=daily_tested_final
india_tested_cum = tested_cum
# nationwide={
#     'dates':dates,
#     'daily_confirmed':daily_confirmed,
#     'daily_confirmed_ma':daily_confirmed_ma,
#     'daily_total':total_confirmed,
#     'daily_positve_rate_ma':daily_pos_rate_ma
# }
# with open('nation.json', 'w') as outfile:
#     json.dump(nationwide, outfile)

In [30]:
print(len(india_total_confirmed ))

134


### State Level Data

In [31]:
population=population.set_index('State')

In [34]:
testing = {}
states = np.unique([x['state'] for x in json.load(open('test.json',))['states_tested_data']])
#df=pd.DataFrame()
csv_dates=[]
csv_states=[]
csv_total_cases=[]
csv_positivity_rate_cumulative=[]
csv_daily_positive_cases=[]
csv_daily_positivity_rate=[]
csv_daily_positive_cases_ma=[]
csv_daily_positivity_rate_ma=[]
csv_test_per_million=[]
csv_daily_tested=[]
csv_cum_tested=[]
for st in states:
#   print(st)
  state_dates = []
  test_per_million=['']*len(dates)
  pos_cum=['']*len(dates)
  pos_rate_cum=['']*len(dates)
  daily_pos=['']*len(dates)
  daily_pos_ma=['']*len(dates)
  daily_tested=['']*len(dates)
  daily_pos_rate=['']*len(dates)
  daily_pos_rate_ma=['']*len(dates)
  tested_cum=['']*len(dates)
  tested_daily=['']*len(dates)
  state_date = [x['updatedon'] for x in filter(lambda v: v['state'] == st,json.load(open('test.json',))['states_tested_data'])]
  #rates = [x['testpositivityrate'] for x in filter(lambda v: v['state'] == st,json.load(open('test.json',))['states_tested_data'])]
  #print(state_date)
  for i in (state_date):
    state_dates.append(convert(i))
  for i in range(len(dates)):
    for j in range(len(state_dates)):
      sum=0
      count=0
      sum1=0
      count1=0
      if (dates[i]==state_dates[j]):
        pos_cum[i] = [x['positive'] for x in filter(lambda v: v['state'] == st,json.load(open('test.json',))['states_tested_data'])][j]
        tested_cum[i] = [x['totaltested'] for x in filter(lambda v: v['state'] == st,json.load(open('test.json',))['states_tested_data'])][j]
        if (j==0):
          daily_pos[i]=pos_cum[j]
          daily_tested[i]=tested_cum[j]
        if(len(pos_cum[i])!=0 and len(pos_cum[i-1])!=0):
          daily_pos[i]=int(pos_cum[i])-int(pos_cum[i-1]) 
          #print(daily_pos[i])
        if(st=='Andaman and Nicobar Islands'):
          daily_pos[i]=pos_cum[i]
        if(len(tested_cum[i])!=0 and len(tested_cum[i-1])!=0):
          daily_tested[i]=(int(tested_cum[i])-int(tested_cum[i-1]))
          test_per_million[i]=int(tested_cum[i])*1000000/int(population["Population"][st])
          if (len(str(daily_pos[i]))!=0 and len(str(tested_cum[i]))!=0):
            pos_rate_cum[i]=int(pos_cum[i])*100/int(tested_cum[i])
          #if(st=='Maharashtra'):
            #print(tested_cum[i],daily_tested[i])
        if (j>6):
          for k in range(0,7):
            if (len(str(daily_pos[i-k]))!=0):
              count+=1
              sum+=int(daily_pos[i-k])
#               print(sum)
          if (count!=0):
            daily_pos_ma[i]=sum/count
          if(len(str(daily_tested[i]))!=0 and daily_tested[i]!=0 and len(str(daily_pos[i]))!=0 ):
            daily_pos_rate[i]=(int(daily_pos[i])*100/int(daily_tested[i]))
        if (j>6):
          for k in range(0,7):
            if (len(str(daily_pos_rate[i-k]))!=0):
              count1+=1
              sum1+=int(daily_pos_rate[i-k])
#               print(sum1)
            if (count1!=0):
             daily_pos_rate_ma[i]=sum1/count1
  if(st=="Maharashtra"):
    print(st,pos_rate_cum)
  for i in range(len(dates)):
    csv_dates.append(dates[i])
    csv_total_cases.append(pos_cum[i])
    csv_states.append(st)
    csv_positivity_rate_cumulative.append(pos_rate_cum[i])
    csv_daily_positive_cases.append(daily_pos[i])
    csv_daily_positivity_rate.append(daily_pos_rate[i])
    csv_daily_positive_cases_ma.append(daily_pos_ma[i])
    csv_daily_positivity_rate_ma.append(daily_pos_rate_ma[i])
    csv_test_per_million.append(test_per_million[i])
    csv_daily_tested.append(daily_tested[i])
    csv_cum_tested.append(tested_cum[i])
  testing[st] = {
                    'dates':dates,
                    'cum_positive_cases':pos_cum,
                    'cum_positivity_rate':pos_rate_cum,
                    'daily_positive_cases':daily_pos,
                    'daily_positivity_rate':daily_pos_rate,
                    'daily_positive_cases_ma': daily_pos_ma,
                    'daily_positivity_rate_ma':daily_pos_rate_ma , 
                    'test_per_million':test_per_million,
                    'daily_tests': daily_tested,
                    'cum_tests': tested_cum    
                  }
testing['India'] = {
                    'dates': india_dates,
                    'cum_positive_cases':india_total_cases,
                    'cum_positivity_rate': india_pos_rate_cum,
                    'daily_positive_cases': india_daily_confirmed,
                    'daily_positivity_rate': india_daily_pos_rate,
                    'daily_positive_cases_ma': india_daily_confirmed_ma,
                    'daily_positivity_rate_ma': india_daily_pos_rate_ma,
                    'test_per_million':india_test_per_million,  
                    'daily_tests':india_daily_tested,
                    'cum_tests':india_tested_cum   
                  }
for i in range (len(dates)):
    csv_dates.append(dates[i])
    csv_states.append('India')
    csv_total_cases.append(india_total_cases[i])
    csv_positivity_rate_cumulative.append(india_pos_rate_cum[i])
    csv_daily_positive_cases.append(india_daily_confirmed[i])
    csv_daily_positivity_rate.append(india_daily_pos_rate[i])
    csv_daily_positive_cases_ma.append(india_daily_confirmed_ma[i])
    csv_daily_positivity_rate_ma.append(india_daily_pos_rate_ma[i])
    csv_test_per_million.append(india_test_per_million[i])
    csv_daily_tested.append(india_daily_tested[i])
    csv_cum_tested.append(india_tested_cum[i])


Andaman and Nicobar Islands
33
66
99
132
38
33
66
99
132
165
34
72
33
66
99
132
165
51
85
123
33
66
99
132
165
198
51
85
123
33
66
99
132
165
198
231
91
142
176
214
33
66
99
132
165
198
231
38
129
180
214
252
33
66
99
132
165
198
231
64
102
193
244
278
316
33
66
99
132
165
198
231
165
229
267
358
409
443
33
66
99
132
165
198
231
68
233
297
335
426
477
33
66
99
132
165
198
231
68
233
297
335
426
33
66
99
132
165
198
231
23
91
256
320
358
449
33
66
99
132
165
198
231
33
56
124
289
353
391
33
66
99
132
165
198
231
11
44
67
135
300
364
33
66
99
132
165
198
231
15
26
59
82
150
315
33
66
99
132
165
198
231
27
42
53
86
109
177
33
66
99
132
165
198
231
11
38
53
64
97
120
33
66
99
132
165
198
231
9
20
47
62
73
106
129
33
66
99
132
165
198
231
16
25
36
63
78
89
122
33
66
99
132
165
198
231
33
49
58
69
96
111
122
33
66
99
132
165
198
231
6
39
55
64
75
102
117
35
68
101
134
167
200
233
23
29
62
78
87
98
125
Andhra Pradesh
9
61
102
129
153
169
0
38
47
99
140
167
191
207
0
31
69
78
130
171
198
222
2

In [35]:
from datetime import datetime
testing['datetime']=str(datetime.now())
with open('positivity_Rate.json', 'w') as outfile:
    json.dump(testing, outfile,indent=4)

In [36]:
df=pd.DataFrame()
df['dates']=csv_dates
df['state']=csv_states
df['cum_positive_cases']=csv_total_cases
df['cum_positivity_rate']=csv_positivity_rate_cumulative
df['daily_positive_cases']=csv_daily_positive_cases
df['daily_positivity_rate']=csv_daily_positivity_rate
df['daily_positive_cases_ma']=csv_daily_positive_cases_ma
df['daily_positivity_rate_ma']=    csv_daily_positivity_rate_ma
df['test_per_million']=csv_test_per_million
df['daily_tested']=csv_daily_tested
df['cum_tested']=csv_cum_tested

In [37]:
df.to_csv('positivity_Rate.csv',index=False)